In [1]:
import os
import sys
import pandas as pd
from google.colab import drive

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
ROOT_DIR = '/content/gdrive/MyDrive/Colab Notebooks'
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'disaster_tweets')
os.listdir(DATA_DIR)

['train.csv',
 'test.csv',
 'model',
 'old_test_train_overlap',
 'old_test_train_split',
 'test_formatted.csv',
 'valid_formatted.csv',
 'train_formatted.csv',
 'submission_20210319075718.csv',
 'submission_20210319081403.csv',
 'submission_20210319083313.csv']

In [4]:
os.listdir(ROOT_DIR)

['data',
 'RLIntro.ipynb',
 'Custom text classification (Kaggle)',
 'requirements.txt',
 'requirementscl.txt',
 'tmp_run.py',
 'TF Text Classification.ipynb']

In [5]:
if ROOT_DIR not in sys.path:
  sys.path.append(ROOT_DIR)

In [6]:
!pip install -r "$ROOT_DIR/requirementscl.txt"

In [7]:
import sys
model_name = "bert-base-uncased"
num_epochs = "4"

args = ['run_tf_text_classification.py']
args.extend([
  "--train_file", os.path.join(DATA_DIR, "train_formatted.csv"),
  "--test_file", os.path.join(DATA_DIR, "test_formatted.csv"),
  "--dev_file", os.path.join(DATA_DIR, "valid_formatted.csv"),
  "--label_column_id", "0",
  "--model_name_or_path", model_name, 
  "--output_dir", os.path.join(DATA_DIR, f"{model_name}_{num_epochs}"), 
  "--num_train_epochs", num_epochs,
  "--per_device_train_batch_size", "16",
  "--per_device_eval_batch_size", "32",
  "--do_train", 
  "--do_eval",
  "--do_predict", 
  "--logging_steps", "460", ### 460 for test 256, 444 for test 512, 476 for all records in training
  "--evaluation_strategy", "steps", 
  "--save_steps", "460", ### WAS: "476", ### 2 times per epoch (476 steps per epoch)
  "--overwrite_output_dir", 
  "--max_seq_length", "128"    
])

print(args)

['run_tf_text_classification.py', '--train_file', '/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/train_formatted.csv', '--test_file', '/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/test_formatted.csv', '--dev_file', '/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/valid_formatted.csv', '--label_column_id', '0', '--model_name_or_path', 'bert-base-uncased', '--output_dir', '/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/bert-base-uncased_4', '--num_train_epochs', '4', '--per_device_train_batch_size', '16', '--per_device_eval_batch_size', '32', '--do_train', '--do_eval', '--do_predict', '--logging_steps', '460', '--evaluation_strategy', 'steps', '--save_steps', '460', '--overwrite_output_dir', '--max_seq_length', '128']


In [8]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2020 The HuggingFace Team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for sequence classification."""
import logging
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import datasets
import numpy as np
import tensorflow as tf

from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
)
from transformers.utils import logging as hf_logging


hf_logging.set_verbosity_info()
hf_logging.enable_default_handler()
hf_logging.enable_explicit_format()


def get_tfds(
    train_file: str,
    eval_file: str,
    test_file: str,
    tokenizer: PreTrainedTokenizer,
    label_column_id: int,
    max_seq_length: Optional[int] = None,
):
    files = {}

    if train_file is not None:
        files[datasets.Split.TRAIN] = [train_file]
    if eval_file is not None:
        files[datasets.Split.VALIDATION] = [eval_file]
    if test_file is not None:
        files[datasets.Split.TEST] = [test_file]

    ds = datasets.load_dataset("csv", data_files=files)
    features_name = list(ds[list(files.keys())[0]].features.keys())
    print(f"Features Name: {features_name}")
    
    label_name = features_name.pop(label_column_id)
    label_list = list(set(ds[list(files.keys())[0]][label_name]))
    label2id = {label: i for i, label in enumerate(label_list)}
    input_names = tokenizer.model_input_names
    transformed_ds = {}

    if len(features_name) == 1:
        for k in files.keys():
            transformed_ds[k] = ds[k].map(
                lambda example: tokenizer.batch_encode_plus(
                    example[features_name[0]], truncation=True, max_length=max_seq_length, padding="max_length"
                ),
                batched=True,
            )
    elif len(features_name) == 2:
        for k in files.keys():
            transformed_ds[k] = ds[k].map(
                lambda example: tokenizer.batch_encode_plus(
                    (example[features_name[0]], example[features_name[1]]),
                    truncation=True,
                    max_length=max_seq_length,
                    padding="max_length",
                ),
                batched=True,
            )

    def gen_train():
        for ex in transformed_ds[datasets.Split.TRAIN]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    def gen_val():
        for ex in transformed_ds[datasets.Split.VALIDATION]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    def gen_test():
        for ex in transformed_ds[datasets.Split.TEST]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    train_ds = (
        tf.data.Dataset.from_generator(
            gen_train,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.TRAIN in transformed_ds
        else None
    )

    if train_ds is not None:
        train_ds = train_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.TRAIN])))

    val_ds = (
        tf.data.Dataset.from_generator(
            gen_val,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.VALIDATION in transformed_ds
        else None
    )

    if val_ds is not None:
        val_ds = val_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.VALIDATION])))

    test_ds = (
        tf.data.Dataset.from_generator(
            gen_test,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.TEST in transformed_ds
        else None
    )

    if test_ds is not None:
        test_ds = test_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.TEST])))

    return train_ds, val_ds, test_ds, label2id


logger = logging.getLogger(__name__)


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.

    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    label_column_id: int = field(metadata={"help": "Which column contains the label"})
    train_file: str = field(default=None, metadata={"help": "The path of the training file"})
    dev_file: Optional[str] = field(default=None, metadata={"help": "The path of the development file"})
    test_file: Optional[str] = field(default=None, metadata={"help": "The path of the test file"})
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )

In [9]:
sys.argv = args

# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TFTrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses()

if (
    os.path.exists(training_args.output_dir)
    and os.listdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(
    "n_replicas: %s, distributed training: %s, 16-bits training: %s",
    training_args.n_replicas,
    bool(training_args.n_replicas > 1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)


[INFO|training_args.py:631] 2021-03-19 10:00:33,055 >> PyTorch: setting up devices
[INFO|training_args.py:555] 2021-03-19 10:00:33,090 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|training_args_tf.py:192] 2021-03-19 10:00:33,094 >> Tensorflow: setting up strategy
03/19/2021 10:00:33 - INFO - __main__ -   n_replicas: 1, distributed training: False, 16-bits training: False
03/19/2021 10:00:33 - INFO - __main__ -   Training/evaluation parameters TFTrainingArguments(output_dir='/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/bert-base-uncased_4', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_ba

In [10]:
# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)


03/19/2021 10:00:33 - INFO - filelock -   Lock 140141410405200 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock
[INFO|file_utils.py:1386] 2021-03-19 10:00:33,693 >> https://huggingface.co/bert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpxk3335ba


[INFO|file_utils.py:1390] 2021-03-19 10:00:33,751 >> storing https://huggingface.co/bert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170
[INFO|file_utils.py:1393] 2021-03-19 10:00:33,753 >> creating metadata file for /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170
03/19/2021 10:00:33 - INFO - filelock -   Lock 140141410405200 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock
[INFO|configuration_utils.py:463] 2021-03-19 10:00:33,757 >> loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/tran

[INFO|file_utils.py:1390] 2021-03-19 10:00:33,859 >> storing https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
[INFO|file_utils.py:1393] 2021-03-19 10:00:33,861 >> creating metadata file for /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
03/19/2021 10:00:33 - INFO - filelock -   Lock 140143859173136 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
03/19/2021 10:00:33 - INFO - filelock -   Lock 140141403597072 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f4

[INFO|file_utils.py:1390] 2021-03-19 10:00:33,974 >> storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
[INFO|file_utils.py:1393] 2021-03-19 10:00:33,976 >> creating metadata file for /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
03/19/2021 10:00:33 - INFO - filelock -   Lock 140141403597072 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
03/19/2021 10:00:34 - INFO - filelock -   Lock 140143859002896 acquired on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a

[INFO|file_utils.py:1390] 2021-03-19 10:00:34,090 >> storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
[INFO|file_utils.py:1393] 2021-03-19 10:00:34,091 >> creating metadata file for /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
03/19/2021 10:00:34 - INFO - filelock -   Lock 140143859002896 released on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
[INFO|tokenization_utils_base.py:1702] 2021-03-19 10:00:34,094 >> loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/trans

In [11]:

train_dataset, eval_dataset, test_ds, label2id = get_tfds(
    train_file=data_args.train_file,
    eval_file=data_args.dev_file,
    test_file=data_args.test_file,
    tokenizer=tokenizer,
    label_column_id=data_args.label_column_id,
    max_seq_length=data_args.max_seq_length,
)


03/19/2021 10:00:34 - WARNING - datasets.builder -   Using custom data configuration default-b765ceac4552832d
03/19/2021 10:00:34 - WARNING - datasets.builder -   Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-b765ceac4552832d/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


Features Name: ['target', 'text']


In [12]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=len(label2id),
    label2id=label2id,
    id2label={id: label for label, id in label2id.items()},
    finetuning_task="text-classification",
    cache_dir=model_args.cache_dir,
)


[INFO|configuration_utils.py:463] 2021-03-19 10:00:35,584 >> loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170
[INFO|configuration_utils.py:499] 2021-03-19 10:00:35,585 >> Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "text-classification",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": 0,
    "1": 1
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": 0,
    "1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type"

In [13]:
with training_args.strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        from_pt=bool(".bin" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )



03/19/2021 10:00:35 - INFO - filelock -   Lock 140141403667920 acquired on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock
[INFO|file_utils.py:1386] 2021-03-19 10:00:35,614 >> https://huggingface.co/bert-base-uncased/resolve/main/tf_model.h5 not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp6sm5ijft


[INFO|file_utils.py:1390] 2021-03-19 10:00:44,580 >> storing https://huggingface.co/bert-base-uncased/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5
[INFO|file_utils.py:1393] 2021-03-19 10:00:44,581 >> creating metadata file for /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5
03/19/2021 10:00:44 - INFO - filelock -   Lock 140141403667920 released on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock
[INFO|modeling_tf_utils.py:1240] 2021-03-19 10:00:44,583 >> loading weights file https://huggingface.co/bert-base-uncased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/tr

[WARNING|modeling_tf_utils.py:1298] 2021-03-19 10:00:46,183 >> All model checkpoint layers were used when initializing TFBertForSequenceClassification.

[WARNING|modeling_tf_utils.py:1302] 2021-03-19 10:00:46,184 >> Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def compute_metrics(p: EvalPrediction) -> Dict:
    preds = np.argmax(p.predictions, axis=1)

    return {"acc": (preds == p.label_ids).mean()}

# Initialize our Trainer
trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)



[INFO|trainer_tf.py:117] 2021-03-19 10:00:46,204 >> You are instantiating a Trainer but W&B is not installed. To use wandb logging, run `pip install wandb; wandb login` see https://docs.wandb.com/huggingface.
[INFO|trainer_tf.py:125] 2021-03-19 10:00:46,205 >> To use comet_ml logging, run `pip/conda install comet_ml` see https://www.comet.ml/docs/python-sdk/huggingface/


In [15]:
print(f"train: {len(train_dataset)}")
print(f"valid: {len(eval_dataset)}")
print(f"test: {len(test_ds)}")

train: 7357
valid: 256
test: 3263


In [16]:
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)


[INFO|trainer_tf.py:528] 2021-03-19 10:00:46,246 >> ***** Running training *****
[INFO|trainer_tf.py:529] 2021-03-19 10:00:46,247 >>   Num examples = 7357
[INFO|trainer_tf.py:531] 2021-03-19 10:00:46,248 >>   Num Epochs = 4
[INFO|trainer_tf.py:532] 2021-03-19 10:00:46,249 >>   Instantaneous batch size per device = 16
[INFO|trainer_tf.py:534] 2021-03-19 10:00:46,250 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer_tf.py:536] 2021-03-19 10:00:46,250 >>   Gradient Accumulation steps = 1
[INFO|trainer_tf.py:537] 2021-03-19 10:00:46,251 >>   Steps per epoch = 460
[INFO|trainer_tf.py:538] 2021-03-19 10:00:46,252 >>   Total optimization steps = 1840


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


03/19/2021 10:00:52 - WARNING - tensorflow -   AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7f75c9fad050>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


03/19/2021 10:00:51 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
03/19/2021 10:00:53 - WARNING - tensorflow -   AutoGraph could not transform <function wrap at 0x7f75e5839c20> and will run it as-is.
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

03/19/2021 10:01:13 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


03/19/2021 10:01:38 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 10:01:38 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:306] 2021-03-19 10:04:47,055 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-03-19 10:04:47,057 >>   Num examples in dataset = 256
[INFO|trainer_tf.py:309] 2021-03-19 10:04:47,058 >>   Num examples in used in evaluation = 256
[INFO|trainer_tf.py:310] 2021-03-19 10:04:47,059 >>   Batch size = 32


03/19/2021 10:04:47 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 10:04:47 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:404] 2021-03-19 10:04:50,791 >> {'eval_loss': 0.4982227087020874, 'eval_acc': 0.796875, 'epoch': 1.0, 'step': 460}
[INFO|trainer_tf.py:404] 2021-03-19 10:04:50,800 >> {'loss': 0.42897147, 'learning_rate': 3.7499995e-05, 'epoch': 1.0, 'step': 460}
[INFO|trainer_tf.py:595] 2021-03-19 10:04:57,945 >> Saving checkpoint for step 460 at /content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/bert-base-uncased_4/checkpoint/ckpt-1
[INFO|trainer_tf.py:306] 2021-03-19 10:08:05,459 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-03-19 10:08:05,461 >>   Num examples in dataset = 256
[INFO|trainer_tf.py:309] 2021-03-19 10:08:05,462 >>   Num examples in used in evaluation = 256
[INFO|trainer_tf.py:310] 2021-03-19 10:08:05,463 >>   Batch size = 32
[INFO|trainer_tf.py:404] 2021-03-19 10:08:07,691 >> {'eval_loss': 0.52798056602

In [17]:
# Evaluation
results = {}
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")

    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")

        for key, value in result.items():
            logger.info("  %s = %s", key, value)
            writer.write("%s = %s\n" % (key, value))

        results.update(result)


03/19/2021 10:14:51 - INFO - __main__ -   *** Evaluate ***
[INFO|trainer_tf.py:306] 2021-03-19 10:14:51,888 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-03-19 10:14:51,889 >>   Num examples in dataset = 256
[INFO|trainer_tf.py:309] 2021-03-19 10:14:51,889 >>   Num examples in used in evaluation = 256
[INFO|trainer_tf.py:310] 2021-03-19 10:14:51,891 >>   Batch size = 32


03/19/2021 10:14:51 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 10:14:52 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:404] 2021-03-19 10:14:57,032 >> {'eval_loss': 0.7061920762062073, 'eval_acc': 0.7734375, 'epoch': 4.0, 'step': 1840}
03/19/2021 10:14:57 - INFO - __main__ -   ***** Eval results *****
03/19/2021 10:14:57 - INFO - __main__ -     eval_loss = 0.7061920762062073
03/19/2021 10:14:57 - INFO - __main__ -     eval_acc = 0.7734375


In [18]:
results

{'eval_acc': 0.7734375, 'eval_loss': 0.7061920762062073}

In [19]:
print(label2id)

{0: 0, 1: 1}


In [20]:
# Prediction
if training_args.do_predict:
    logger.info("*** predictions ***")
    preds = trainer.predict(test_ds)

    logger.info("*** RESUTS: ***")
    logger.info(preds)
    logger.info("*** :RESUTS ***")


03/19/2021 10:14:57 - INFO - __main__ -   *** predictions ***
[INFO|trainer_tf.py:306] 2021-03-19 10:14:57,085 >> ***** Running Prediction *****
[INFO|trainer_tf.py:307] 2021-03-19 10:14:57,088 >>   Num examples in dataset = 3263
[INFO|trainer_tf.py:310] 2021-03-19 10:14:57,090 >>   Batch size = 32


03/19/2021 10:15:25 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 10:15:25 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
03/19/2021 10:15:27 - INFO - __main__ -   *** RESUTS: ***
03/19/2021 10:15:27 - INFO - __main__ -   PredictionOutput(predictions=array([[-2.3198555,  2.3989658],
       [-2.66637  ,  2.7750852],
       [-2.237695 ,  2.2953863],
       ...,
       [-3.195193 ,  3.3258626],
       [-1.503033 ,  1.6274265],
       [-2.1759598,  2.3290763]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'eval_loss': 1.9748593498678768, 'eval_acc': 0.5807539074471345})
03/19/2021 10:15:27 - INFO - __main__ -   *** :RESUTS ***


In [21]:
decoded_preds = np.argmax(preds.predictions, axis=1)

In [22]:
decoded_preds[10:30]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1])

In [23]:
org_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [24]:
org_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [25]:
org_test['target'] = decoded_preds

In [26]:
org_test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [27]:
from datetime import datetime

In [28]:
timestamp=str(datetime.now()).replace(' ','').replace(':','').replace('-','').split('.')[0]

In [29]:
org_test[['id', 'target']].to_csv(os.path.join(DATA_DIR, f'./submission_{timestamp}.csv'), index=False)